In [8]:
import pandas as pd
import itertools
import math

# Function definitions remain the same...

# Read the CSV file
df = pd.read_csv('Next30_Odds.csv')

# List to hold bets on favorites with adjusted odds range
favorite_bets = []

# Process each row to identify favorites
for index, row in df.iterrows():
    try:
        odds1 = float(row['odds1'])
        odds2 = float(row['odds2'])
        team1 = row['team1']
        team2 = row['team2']
        event = f"{row['sport']} {row['event_time']}"
    except (ValueError, KeyError, TypeError):
        continue  # Skip rows with invalid data

    # Identify favorite and underdog
    if odds1 < odds2:
        favorite_odds = odds1
        underdog_odds = odds2
        favorite_team = team1
    else:
        favorite_odds = odds2
        underdog_odds = odds1
        favorite_team = team2

    # Adjusted odds range (-2000 to -200)
    if favorite_odds < 0 and -2000 <= favorite_odds <= -200:
        # Calculate implied probabilities
        favorite_implied_prob = american_to_implied_prob(favorite_odds)
        underdog_implied_prob = american_to_implied_prob(underdog_odds)
        # Calculate discrepancy as ratio
        if underdog_implied_prob > 0:
            discrepancy = favorite_implied_prob / underdog_implied_prob
        else:
            discrepancy = float('inf')
        # Store the bet with discrepancy
        favorite_bets.append({
            'event': event,
            'team': favorite_team,
            'odds': favorite_odds,
            'decimal_odds': american_to_decimal(favorite_odds),
            'implied_prob': favorite_implied_prob,
            'discrepancy': discrepancy
        })

# Check the number of favorite bets
print(f"Number of favorite bets: {len(favorite_bets)}")

# Sort favorite bets by discrepancy (descending) and limit to top K bets
K = 50  # Adjust K as needed
favorite_bets = sorted(favorite_bets, key=lambda x: x['discrepancy'], reverse=True)[:K]

# Pre-calculate the logarithms of decimal odds and implied probabilities
for bet in favorite_bets:
    bet['log_decimal_odds'] = math.log(bet['decimal_odds'])
    bet['log_implied_prob'] = math.log(bet['implied_prob'])

# Generate all combinations of bets (parlays)
parlays = []

for r in range(2, 6):  # Consider parlays of size 2 to 5
    combinations = itertools.combinations(favorite_bets, r)
    for combo in combinations:
        # Ensure all events are unique
        event_names = [bet['event'] for bet in combo]
        if len(set(event_names)) < len(event_names):
            continue  # Skip combinations with duplicate events

        # Calculate combined decimal odds and implied probability
        combined_decimal_odds = math.exp(sum(bet['log_decimal_odds'] for bet in combo))
        combined_prob = math.exp(sum(bet['log_implied_prob'] for bet in combo))
        min_discrepancy = min(bet['discrepancy'] for bet in combo)
        events = [bet['event'] for bet in combo]
        teams = [bet['team'] for bet in combo]
        odds_list = [bet['odds'] for bet in combo]
        # Convert combined decimal odds to American odds
        combined_american_odds = decimal_to_american(combined_decimal_odds)
        # Adjust combined odds range
        if 100 <= combined_american_odds <= 300:
            # Calculate expected value
            payout = combined_decimal_odds - 1  # Profit per $1 bet
            expected_value = (combined_prob * payout) - ((1 - combined_prob) * 1)
            parlays.append({
                'parlay': list(zip(events, teams, odds_list)),
                'combined_odds': combined_american_odds,
                'implied_probability': combined_prob,
                'expected_value': expected_value,
                'min_discrepancy': min_discrepancy
            })

# Check if any parlays were generated
if not parlays:
    print("No parlays generated with the current criteria.")
else:
    # Sort parlays from most likely to least likely based on implied probability and minimum discrepancy
    sorted_parlays = sorted(parlays, key=lambda x: (-x['implied_probability'], -x['min_discrepancy']))

    # Output the results to a file
    with open('parlay_results.txt', 'w') as f:
        for parlay in sorted_parlays:
            f.write("Parlay:\n")
            for event, team, odds in parlay['parlay']:
                f.write(f"  Event: {event}, Team: {team}, Odds: {odds}\n")
            f.write(f"Combined Odds (American): +{parlay['combined_odds']:.2f}\n")
            f.write(f"Implied Probability: {parlay['implied_probability'] * 100:.2f}%\n")
            f.write(f"Minimum Discrepancy: {parlay['min_discrepancy']:.2f}\n")
            f.write(f"Expected Value per $1 bet: ${parlay['expected_value']:.2f}\n")
            f.write("-----\n")

    print("Parlay results have been written to 'parlay_results.txt'.")

Number of favorite bets: 148
Parlay results have been written to 'parlay_results.txt'.
